In [1]:
%run __init__.ipynb

Successfully connected to MongoDB


In [2]:
%%time

#--- Check if the data are already updated
def init_script():
    stop = False
    file_path = os.path.join(OUTPUT_FOLDER,'WORLD_DB_TREND.csv')
    if path.exists(file_path):
        df_check = pd.read_csv(file_path,
                               sep=';',
                               low_memory=False)
        
        date_check = df_check[df_check['SCENARIO'] == YESTERDAY.strftime('%d/%m/%Y')]
        if len(date_check) == 0:
            %run "covid-19_world.ipynb"
        else:
            raise Exception("Script already updated !")
    else:
        %run "covid-19_world.ipynb"
    
init_script()

Successfully connected to MongoDB
WORLD_CONFIRMED successfully saved in WORLD_CONFIRMED
Dataframe WORLD_CONFIRMED successfully save in database covid-19 in MongoDB. Time: --- 0.1484668254852295 secnds ---
WORLD_DEATHS successfully saved in WORLD_DEATHS
Dataframe WORLD_DEATHS successfully save in database covid-19 in MongoDB. Time: --- 0.2116997241973877 secnds ---
WORLD_RECOVERED successfully saved in WORLD_RECOVERED
Dataframe WORLD_RECOVERED successfully save in database covid-19 in MongoDB. Time: --- 0.15952587127685547 secnds ---
US_CONFIRMED successfully saved in US_CONFIRMED
Dataframe US_CONFIRMED successfully save in database covid-19 in MongoDB. Time: --- 6.638959646224976 secnds ---
US_DEATHS successfully saved in US_DEATHS
Dataframe US_DEATHS successfully save in database covid-19 in MongoDB. Time: --- 6.482724905014038 secnds ---
Error connecting to https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/11-09-2020.csv.


UsageError: Line magic function `%stop` not found.


WORLD_DB_CONSO successfully saved in WORLD_DB_CONSO
CPU times: user 1.28 s, sys: 16.3 ms, total: 1.29 s
Wall time: 1.49 s
WORLD_DB_ALL successfully saved in WORLD_DB_ALL
CPU times: user 24.8 s, sys: 799 ms, total: 25.6 s
Wall time: 27.7 s


<timed exec>:23: FutureWarning: This dataframe has a column name that matches the 'value_name' column name of the resultiing Dataframe. In the future this will raise an error, please set the 'value_name' parameter of DataFrame.melt to a unique name.


WORLD_DB_TREND successfully saved in WORLD_DB_TREND
CPU times: user 36.4 s, sys: 773 ms, total: 37.2 s
Wall time: 42.8 s
Column 11/09/20 does not exist in CONFIRMED.

Column 11/09/20 does not exist in CONFIRMED.

Column 11/09/20 does not exist in DEATHS.

Column 11/09/20 does not exist in DEATHS.

Column 11/09/20 does not exist in RECOVERED.

No data in Daily

No data in Daily

No data in Daily

Data check 2020-11-09: TimeSeries - Daily
Confirmed: 0 - 0 = 0
Deaths: 0 - 0 = 0
Recovered: 0 - 0 = 0
Active cases: 0 - 0 = 0
Fatality rate: 0 - 0 = 0
Recovery rate: 0 - 0 = 0
CPU times: user 1min 15s, sys: 2.61 s, total: 1min 18s
Wall time: 1min 37s


In [3]:
%%time

#--- Load data source
db_conso = get_datasource('WORLD_DB_CONSO')
db_all = get_datasource('WORLD_DB_ALL')
db_trend = get_datasource('WORLD_DB_TREND')

CPU times: user 6.05 s, sys: 980 ms, total: 7.03 s
Wall time: 8.39 s


In [4]:
def df_to_csv_mongo(df,df_name):
    if not df.empty:
        # Save in CSV
        df_save(df, df_name,'csv')
        
        # Save in MongoDB
        if USE_MONGO:
            naas_drivers.mongo.send(df, df_name, DB_APP, True)
    else:
        print(f'DataFrame {df_name} is empty !')

In [5]:
%%time

#Dashboard value dynamic template
def dashboard_value(df, kpi, domain_num):
    filters = ['WORLDWIDE','Europe','Asia','Africa','North America','South America','Oceania','Other','To be affected']
    domain = df[(df['ENTITY_GROUPS'].isin(filters)) & (df['KPI'].isin([kpi]))]
    domain = pd.DataFrame({
        'ENTITY': domain['ENTITY'],
        'ENTITY_GROUPS': domain['ENTITY_GROUPS'],
        'SCENARIO': domain['SCENARIO'],
        'DATE_ORDER': domain['DATE_ORDER'],
        'LABEL': domain['KPI'],
        'VALUE': domain['VALUE'],
        'VALUE_D-1': domain['VALUE_D-1'],
        'VAR': domain['VARV'],
        'VARP': domain['VARP'],
        'UNIT_VALUE': domain['UNIT_VALUE'],
        'UNIT_VAR': domain['UNIT_VAR'],
        'UNIT_VARP': domain['UNIT_VARP'],
        'PRECISION_VALUE': domain['PRECISION_VALUE'],
        'PRECISION_VAR': domain['PRECISION_VAR'],
        'PRECISION_VARP': domain['PRECISION_VARP'],
        'LAST_UPDATE': datetime.now().strftime('%d/%m/%Y %H:%M:%S'),
    })
    domain = domain.sort_values(by=['ENTITY','SCENARIO'],ascending=False)
    domain = optimize(domain, ['LAST_UPDATE'])
    df_to_csv_mongo(domain,domain_num)
    print(DB_APP, domain_num)
    df_collection = MONGO_DB[DB_APP][domain_num]
    df_collection.create_index([("ENTITY", DESCENDING),("SCENARIO", DESCENDING)])
    return domain

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.2 µs


In [6]:
#Duplicate to rank countries WORLDWIDE
def get_ww(df): 
    filters = ['Europe','Asia','Africa','North America','South America','Oceania','Other','To be affected']
    df_ww = df.copy()[(df['ENTITY'].isin(filters))] 
    df_ww['ENTITY'] = "WORLDWIDE"           
    df = df.append(df_ww)
    return df

In [7]:
#Dashboard leaderboard template
def dashboard_leaderboard(df,kpi,domain_num):
    domain = df[(df['KPI'].isin([kpi]))]
    domain = pd.DataFrame({
        'ENTITY': domain['ENTITY_GROUPS'],
        'SCENARIO': domain['SCENARIO'],
        'DATE_ORDER': domain['DATE_ORDER'],
        'LABEL': domain['ENTITY'],
        'VALUE': domain['VALUE'],
        'VALUE_D-1': domain['VALUE_D-1'],
        'VAR': domain['VARV'],
        'VARP': domain['VARP'],
        'UNIT_VALUE': domain['UNIT_VALUE'],
        'UNIT_VAR': domain['UNIT_VAR'],
        'UNIT_VARP': domain['UNIT_VARP'],
        'PRECISION_VALUE': domain['PRECISION_VALUE'],
        'PRECISION_VAR': domain['PRECISION_VAR'],
        'PRECISION_VARP': domain['PRECISION_VARP'],
        'LAST_UPDATE': datetime.now().strftime('%d/%m/%Y %H:%M:%S'),
    })
    domain = domain.sort_values(by=['ENTITY','SCENARIO'],ascending=False)
    if domain_num == '010' or domain_num == '011' or domain_num == '012':
        domain = get_ww(domain)
    df_to_csv_mongo(domain,domain_num)
    df_collection = MONGO_DB[DB_APP][domain_num]
    df_collection.create_index([("ENTITY", DESCENDING),("SCENARIO", DESCENDING)])
    df_collection = optimize(df_collection, ['LAST_UPDATE'])
    return domain

In [8]:
%%time

#-- Date selector : ENTITIES
def get_entities(df):
    cols_to_rename = {"ENTITY": "ENTITIES", "ENTITY_GROUPS": "GROUPS"}
    filters = ['WORLDWIDE','Europe','Asia','Africa','North America','South America','Oceania','Other','To be affected']
    df = df[df['ENTITY_GROUPS'].isin(filters)].rename(index=str, columns=cols_to_rename)[['ENTITIES','GROUPS']].drop_duplicates().sort_values(by="ENTITIES", ascending=True).reset_index(drop=True)
    df['ORDER'] = df.index + 8
    df.loc[df['ENTITIES'] == 'WORLDWIDE', 'ORDER'] = 0
    df.loc[df['ENTITIES'] == 'Europe', 'ORDER'] = 1
    df.loc[df['ENTITIES'] == 'Asia', 'ORDER'] = 2
    df.loc[df['ENTITIES'] == 'Africa', 'ORDER'] = 3
    df.loc[df['ENTITIES'] == 'North America', 'ORDER'] = 4
    df.loc[df['ENTITIES'] == 'South America', 'ORDER'] = 5
    df.loc[df['ENTITIES'] == 'Oceania', 'ORDER'] = 6
    df.loc[df['ENTITIES'] == 'Other', 'ORDER'] = 7
    df.loc[df['ENTITIES'] == 'To be affected', 'ORDER'] = 8
    
    df.loc[df['ENTITIES'] == 'WORLDWIDE', 'ORDER_GROUPS'] = 0
    df.loc[df['ENTITIES'] == 'Europe', 'ORDER_GROUPS'] = 1
    df.loc[df['ENTITIES'] == 'Asia', 'ORDER_GROUPS'] = 2
    df.loc[df['ENTITIES'] == 'Africa', 'ORDER_GROUPS'] = 3
    df.loc[df['ENTITIES'] == 'North America', 'ORDER_GROUPS'] = 4
    df.loc[df['ENTITIES'] == 'South America', 'ORDER_GROUPS'] = 5
    df.loc[df['ENTITIES'] == 'Oceania', 'ORDER_GROUPS'] = 6
    df.loc[df['ENTITIES'] == 'Other', 'ORDER_GROUPS'] = 7
    df.loc[df['ENTITIES'] == 'To be affected', 'ORDER_GROUPS'] = 8

    df.loc[df['GROUPS'] == 'Europe', 'ORDER_GROUPS'] = 1
    df.loc[df['GROUPS'] == 'Asia', 'ORDER_GROUPS'] = 2
    df.loc[df['GROUPS'] == 'Africa', 'ORDER_GROUPS'] = 3
    df.loc[df['GROUPS'] == 'North America', 'ORDER_GROUPS'] = 4
    df.loc[df['GROUPS'] == 'South America', 'ORDER_GROUPS'] = 5
    df.loc[df['GROUPS'] == 'Oceania', 'ORDER_GROUPS'] = 6
    df.loc[df['GROUPS'] == 'Other', 'ORDER_GROUPS'] = 7
    df.loc[df['GROUPS'] == 'To be affected', 'ORDER_GROUPS'] = 8
    
    df.loc[df['GROUPS'] == 'WORLDWIDE','ENTITY_DISPLAY'] = df['ENTITIES']
    df.loc[df['GROUPS'] != 'WORLDWIDE','ENTITY_DISPLAY'] = '--- ' + df['ENTITIES']
    df = optimize(df)
    return df.sort_values(by=['ORDER_GROUPS','ORDER'], ascending=True)

entities = get_entities(db_conso)
df_to_csv_mongo(entities,'ENTITIES')
# entities

ENTITIES successfully saved in ENTITIES
Dataframe ENTITIES successfully save in database app-wsr in MongoDB. Time: --- 0.0238039493560791 secnds ---
CPU times: user 60.1 ms, sys: 3.63 ms, total: 63.7 ms
Wall time: 147 ms


In [9]:
%%time

#-- Date selector : SCENARIOS
def get_scenarios(df):
    df = df[['SCENARIO','DATE_ORDER']].drop_duplicates()
    df['SCENARIO_DIPLAY'] = pd.to_datetime(df['SCENARIO'], format='%d/%m/%Y').dt.strftime('%d %b %Y')
    df['SCENARIO_TODAY'] = (pd.to_datetime(df['SCENARIO'], format='%d/%m/%Y') + timedelta(days = 1)).dt.strftime('%d/%m/%Y')
    df['SCENARIO_TODAY_DIPLAY'] = pd.to_datetime(df['SCENARIO_TODAY'], format='%d/%m/%Y').dt.strftime('%d %b %Y')
    df = optimize(df)
    return df.sort_values(by='DATE_ORDER', ascending=False)

scenarios = get_scenarios(db_all)
df_to_csv_mongo(scenarios,'SCENARIOS')
# scenarios

SCENARIOS successfully saved in SCENARIOS
Dataframe SCENARIOS successfully save in database app-wsr in MongoDB. Time: --- 0.03311491012573242 secnds ---
CPU times: user 95.6 ms, sys: 0 ns, total: 95.6 ms
Wall time: 159 ms


In [10]:
%%time

#--- DASHBOARD ---
#-- VALUE DYNAMIC KPIS
domain001 = dashboard_value(db_all,'Confirmed','001')
domain002 = dashboard_value(db_all,'Deaths','002')
domain003 = dashboard_value(db_all,'Recovered','003')
domain004 = dashboard_value(db_all,'Active cases','004')
domain005 = dashboard_value(db_all,"Fatality Rate",'005')
domain006 = dashboard_value(db_all,"Recovery Rate",'006')

001 successfully saved in 001
Dataframe 001 successfully save in database app-wsr in MongoDB. Time: --- 3.0506751537323 secnds ---
app-wsr 001
002 successfully saved in 002
Dataframe 002 successfully save in database app-wsr in MongoDB. Time: --- 2.948002576828003 secnds ---
app-wsr 002
003 successfully saved in 003
Dataframe 003 successfully save in database app-wsr in MongoDB. Time: --- 3.069716215133667 secnds ---
app-wsr 003
004 successfully saved in 004
Dataframe 004 successfully save in database app-wsr in MongoDB. Time: --- 2.790334463119507 secnds ---
app-wsr 004
005 successfully saved in 005
Dataframe 005 successfully save in database app-wsr in MongoDB. Time: --- 2.7838499546051025 secnds ---
app-wsr 005
006 successfully saved in 006
Dataframe 006 successfully save in database app-wsr in MongoDB. Time: --- 2.6809375286102295 secnds ---
app-wsr 006
CPU times: user 14.9 s, sys: 247 ms, total: 15.1 s
Wall time: 24.8 s


In [11]:
%%time

#-- LEADERBOARD BY COUNTRIES
db_c = db_all[(db_all['ENTITY_GROUPS'] != 'WORLDWIDE')]
domain010 = dashboard_leaderboard(db_c,'Confirmed','010')
domain011 = dashboard_leaderboard(db_c,'Deaths','011')
domain012 = dashboard_leaderboard(db_c,'Recovered','012')

010 successfully saved in 010
Dataframe 010 successfully save in database app-wsr in MongoDB. Time: --- 46.58505177497864 secnds ---
011 successfully saved in 011
Dataframe 011 successfully save in database app-wsr in MongoDB. Time: --- 43.81312322616577 secnds ---
012 successfully saved in 012
Dataframe 012 successfully save in database app-wsr in MongoDB. Time: --- 44.07226896286011 secnds ---
CPU times: user 42.9 s, sys: 2.11 s, total: 45.1 s
Wall time: 2min 23s


In [12]:
%%time

#-- Covid-19 - TRENDS
def domain_101(df,domain_num):
#     filters = ['Confirmed','Deaths','Recovered','Active cases','Deaths ratio','Recovered ratio']
    filters = ['Confirmed','Deaths','Recovered','Active cases']
    domain = df[df['KPI'].isin(filters)]
    domain = pd.DataFrame({
        'ENTITY': domain['ENTITY'],
        'ENTITY_GROUPS': domain['ENTITY_GROUPS'],
        'SCENARIO': domain['SCENARIO'],
        'DATE_ORDER': domain['DATE_ORDER'],
        'GROUPS': domain['KPI'],
        'BOTTOM_FILTER': domain['DATE_SCENARIO'],
        'UPPER_FILTER_M': domain['METRIC'],
        'VALUE': domain['VALUE'],
        'UNIT_VALUE': domain['UNIT'],
        'PRECISION': domain['PRECISION'],
        'LAST_UPDATE': datetime.now().strftime('%d/%m/%Y %H:%M:%S'),
    })

    #Deconfinement France
    groups_name = 'Deconfinement'
    df_france = domain.copy()[(domain['ENTITY'] == 'France') & (domain['GROUPS'] == 'Confirmed') & (domain['UPPER_FILTER_M'] == 'VARV')]
    df_france.loc[:,'GROUPS'] = groups_name
    df_france.loc[:,'VALUE'] = 3000
    domain = pd.concat([domain,df_france],axis=0)
    
    #Rename upper filter
    domain.loc[domain['UPPER_FILTER_M'] == 'VALUE','UPPER_FILTER_M'] = 'EVOLUTION'
    domain.loc[(domain['UPPER_FILTER_M'] == 'VARV') & (domain['GROUPS'].isin(['Confirmed','Deaths','Recovered','Active cases',groups_name])),'UPPER_FILTER_M'] = 'DAILY VARIATION (CASES)'
#     domain.loc[(domain['UPPER_FILTER_M'] == 'VARV') & (domain['GROUPS'].isin(['Deaths ratio','Recovered ratio'])),'UPPER_FILTER_M'] = 'DAILY VARIATION (PTS)'
    domain.loc[(domain['UPPER_FILTER_M'] == 'VARP') & (domain['GROUPS'].isin(['Confirmed','Deaths','Recovered','Active cases',groups_name])),'UPPER_FILTER_M'] = 'DAILY VARIATION (%)'
#     domain.loc[(domain['UPPER_FILTER_M'] == 'VARP') & (domain['GROUPS'].isin(['Deaths ratio','Recovered ratio'])),'UPPER_FILTER_M'] = ''
    
    df_to_csv_mongo(domain,domain_num)
    df_collection = MONGO_DB[DB_APP][domain_num]
    df_collection.create_index([("ENTITY", DESCENDING), ("UPPER_FILTER_M", DESCENDING), ("BOTTOM_FILTER", DESCENDING)])
    domain = optimize(domain, ['LAST_UPDATE'])
    return domain

domain101 = domain_101(db_trend,'101')
# domain101

101 successfully saved in 101
Dataframe 101 successfully save in database app-wsr in MongoDB. Time: --- 131.1746120452881 secnds ---
CPU times: user 1min 13s, sys: 3.5 s, total: 1min 17s
Wall time: 2min 45s


In [13]:
db_trend.info()
domain101.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3181248 entries, 0 to 3181247
Data columns (total 12 columns):
 #   Column         Dtype  
---  ------         -----  
 0   ENTITY_GROUPS  object 
 1   ENTITY         object 
 2   SCENARIO       object 
 3   DATE           object 
 4   DATE_ORDER     int64  
 5   KPI            object 
 6   METRIC         object 
 7   VALUE          float64
 8   DATE_SCENARIO  object 
 9   KPI_ORDER      float64
 10  UNIT           object 
 11  PRECISION      object 
dtypes: float64(2), int64(1), object(9)
memory usage: 291.3+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2121504 entries, 0 to 3175198
Data columns (total 11 columns):
 #   Column          Dtype         
---  ------          -----         
 0   ENTITY          category      
 1   ENTITY_GROUPS   category      
 2   SCENARIO        category      
 3   DATE_ORDER      int32         
 4   GROUPS          category      
 5   BOTTOM_FILTER   category      
 6   UPPER_FILTER_M  category 

In [14]:
domain101

,ENTITY,ENTITY_GROUPS,SCENARIO,DATE_ORDER,GROUPS,BOTTOM_FILTER,UPPER_FILTER_M,VALUE,UNIT_VALUE,PRECISION,LAST_UPDATE
0,Afghanistan,Afghanistan,22/01/2020,20200122,Confirmed,Since beginning,EVOLUTION,0.0,cases,",.0f",2020-10-11 02:53:51
1,Afghanistan,Afghanistan,23/01/2020,20200123,Confirmed,Since beginning,EVOLUTION,0.0,cases,",.0f",2020-10-11 02:53:51
2,Afghanistan,Afghanistan,24/01/2020,20200124,Confirmed,Since beginning,EVOLUTION,0.0,cases,",.0f",2020-10-11 02:53:51
3,Afghanistan,Afghanistan,25/01/2020,20200125,Confirmed,Since beginning,EVOLUTION,0.0,cases,",.0f",2020-10-11 02:53:51
4,Afghanistan,Afghanistan,26/01/2020,20200126,Confirmed,Since beginning,EVOLUTION,0.0,cases,",.0f",2020-10-11 02:53:51
...,...,...,...,...,...,...,...,...,...,...,...
3156262,France,France,12/10/2020,20201012,Deconfinement,Last 30 days,DAILY VARIATION (CASES),3000.0,cases,"+,.0f",2020-10-11 02:53:51
3165692,France,Europe,11/10/2020,20201011,Deconfinement,Last 30 days,DAILY VARIATION (CASES),3000.0,cases,"+,.0f",2020-10-11 02:53:51
3165730,France,France,11/10/2020,20201011,Deconfinement,Last 30 days,DAILY VARIATION (CASES),3000.0,cases,"+,.0f",2020-10-11 02:53:51
3175160,France,Europe,10/10/2020,20201010,Deconfinement,Last 30 days,DAILY VARIATION (CASES),3000.0,cases,"+,.0f",2020-10-11 02:53:51


In [15]:
%%time

#-- Covid-19 - TRENDS BY COUNTRIES
def domain_102(df,domain_num):
    domain = df.copy()
    max_date = domain['DATE_ORDER'].max()
    
    filters =  ['Europe','Asia','Africa','North America','South America','Oceania','Other','To be affected']
    countries_df = domain[(domain['ENTITY_GROUPS'].isin(filters)) & (domain['DATE_ORDER'] == max_date) & (domain['GROUPS'] == 'Confirmed') & (domain['BOTTOM_FILTER'] == 'Since beginning')].sort_values(by= 'VALUE', ascending=False).drop_duplicates().reset_index(drop=True)[:10]#['ENTITY'].drop_duplicates().values.tolist()[:10]
    countries_df['ENTITY_ORDER'] = countries_df.index
    countries_df = countries_df[['ENTITY','ENTITY_ORDER']]
    
    countries = countries_df['ENTITY'].values.tolist()
    
    domain = domain[(domain['ENTITY_GROUPS'] != "WORLDWIDE") & (domain['ENTITY'].isin(countries))]
    domain = pd.merge(domain,countries_df, on=['ENTITY'], how='left')

    df_to_csv_mongo(domain,domain_num)
    df_collection = MONGO_DB[DB_APP][domain_num]
    df_collection.create_index([("GROUPS", ASCENDING), ("UPPER_FILTER_M", DESCENDING), ("BOTTOM_FILTER", DESCENDING)])
    domain = optimize(domain, ['LAST_UPDATE'])
    return domain
    
domain_102(domain101,'102')
# domain102 = domain_102(domain101,'102')
# domain102

102 successfully saved in 102
Dataframe 102 successfully save in database app-wsr in MongoDB. Time: --- 3.603322744369507 secnds ---
CPU times: user 2.58 s, sys: 23.2 ms, total: 2.6 s
Wall time: 5.21 s


,ENTITY,ENTITY_GROUPS,SCENARIO,DATE_ORDER,GROUPS,BOTTOM_FILTER,UPPER_FILTER_M,VALUE,UNIT_VALUE,PRECISION,LAST_UPDATE,ENTITY_ORDER
0,Argentina,Argentina,22/01/2020,20200122,Confirmed,Since beginning,EVOLUTION,0.0,cases,",.0f",2020-10-11 02:53:51,6
1,Argentina,Argentina,23/01/2020,20200123,Confirmed,Since beginning,EVOLUTION,0.0,cases,",.0f",2020-10-11 02:53:51,6
2,Argentina,Argentina,24/01/2020,20200124,Confirmed,Since beginning,EVOLUTION,0.0,cases,",.0f",2020-10-11 02:53:51,6
3,Argentina,Argentina,25/01/2020,20200125,Confirmed,Since beginning,EVOLUTION,0.0,cases,",.0f",2020-10-11 02:53:51,6
4,Argentina,Argentina,26/01/2020,20200126,Confirmed,Since beginning,EVOLUTION,0.0,cases,",.0f",2020-10-11 02:53:51,6
...,...,...,...,...,...,...,...,...,...,...,...,...
81307,France,France,12/10/2020,20201012,Deconfinement,Last 30 days,DAILY VARIATION (CASES),3000.0,cases,"+,.0f",2020-10-11 02:53:51,3
81308,France,Europe,11/10/2020,20201011,Deconfinement,Last 30 days,DAILY VARIATION (CASES),3000.0,cases,"+,.0f",2020-10-11 02:53:51,3
81309,France,France,11/10/2020,20201011,Deconfinement,Last 30 days,DAILY VARIATION (CASES),3000.0,cases,"+,.0f",2020-10-11 02:53:51,3
81310,France,Europe,10/10/2020,20201010,Deconfinement,Last 30 days,DAILY VARIATION (CASES),3000.0,cases,"+,.0f",2020-10-11 02:53:51,3


In [16]:
%%time

#-- Covid-19 - RANKING BY COUNTRIES
#-> Referentials
ref_continent = pd.read_excel(REF_WSR_PATH, sheet_name='REF_CONTINENT')
def domain_103(df,ref,domain_num):
    domain = df.copy()
    cols_to_rename = {"LABEL": "KPI","ENTITY": "LABEL", "ENTITY_GROUPS":"ENTITY", "VARV":"VAR"}
    domain = get_ww(domain.drop(['DATE','UNIT_VAR','UNIT_VARP','PRECISION_VALUE','PRECISION_VAR','PRECISION_VARP'],axis=1).rename(index=str, columns=cols_to_rename).reset_index(drop=True))
    domain = pd.merge(domain, ref, left_on=['LABEL'], right_on=['COUNTRY_REGION'], how='left').drop(['CONTINENT','COUNTRY_REGION'],axis=1).reset_index(drop=True)

    df_to_csv_mongo(domain,domain_num)
    df_collection = MONGO_DB[DB_APP][domain_num]
    df_collection.create_index([("ENTITY", DESCENDING), ("SCENARIO", DESCENDING), ("LABEL", DESCENDING), ("KPI", ASCENDING), ("VALUE", DESCENDING)])
    domain = optimize(domain, ['LAST_UPDATE'])
    return domain

domain_103(db_all,ref_continent,'103')
# domain103 = domain_103(db_all,ref_continent,'103')
# domain103

103 successfully saved in 103
Dataframe 103 successfully save in database app-wsr in MongoDB. Time: --- 113.76619458198547 secnds ---
CPU times: user 54.2 s, sys: 1.65 s, total: 55.9 s
Wall time: 2min 18s


,ENTITY,LABEL,KPI,VALUE,VALUE_D-1,VAR,VARP,SCENARIO,DATE_ORDER,UNIT_VALUE,WORLDMAP,WORLDMAP_CS,CONTINENT_CS
0,Afghanistan,Afghanistan,Confirmed,0.00000,0.00000,0.0,NaN,22/01/2020,20200122,cases,Afghanistan,Afghanistan,Asia
1,Afghanistan,Afghanistan,Confirmed,0.00000,0.00000,0.0,NaN,23/01/2020,20200123,cases,Afghanistan,Afghanistan,Asia
2,Afghanistan,Afghanistan,Confirmed,0.00000,0.00000,0.0,NaN,24/01/2020,20200124,cases,Afghanistan,Afghanistan,Asia
3,Afghanistan,Afghanistan,Confirmed,0.00000,0.00000,0.0,NaN,25/01/2020,20200125,cases,Afghanistan,Afghanistan,Asia
4,Afghanistan,Afghanistan,Confirmed,0.00000,0.00000,0.0,NaN,26/01/2020,20200126,cases,Afghanistan,Afghanistan,Asia
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254427,WORLDWIDE,Solomon Islands,Recovery Rate,30.76923,30.76923,0.0,NaN,04/11/2020,20201104,%,NaN,NaN,NaN
1254428,WORLDWIDE,Solomon Islands,Recovery Rate,30.76923,30.76923,0.0,NaN,05/11/2020,20201105,%,NaN,NaN,NaN
1254429,WORLDWIDE,Solomon Islands,Recovery Rate,30.76923,30.76923,0.0,NaN,06/11/2020,20201106,%,NaN,NaN,NaN
1254430,WORLDWIDE,Solomon Islands,Recovery Rate,30.76923,30.76923,0.0,NaN,07/11/2020,20201107,%,NaN,NaN,NaN


In [17]:
%%time

#-- Covid-19 - DAILY TREND
def domain_106(df,domain_num):
    #-- Exclude Rates & Date
    date_list = df['DATE_ORDER'].drop_duplicates().sort_values(ascending=False).values.tolist()[:15]
    df = df[(~df['KPI'].isin(['Fatality Rate',"Recovery Rate"])) &
            (df['DATE_ORDER'].isin(date_list))]
    
    #-- Calculate Value
    filters = ['WORLDWIDE','Europe','Asia','Africa','North America','South America','Oceania','Other','To be affected']
    domain_val = df.copy()[['ENTITY_GROUPS','ENTITY','SCENARIO','KPI','VALUE','VALUE_D-1']][df['ENTITY_GROUPS'].isin(filters)]#[df['ENTITY_GROUPS'] == 'WORLDWIDE']
    cols_to_keep = ['ENTITY_GROUPS','ENTITY','SCENARIO','KPI']
    domain_val = domain_val.melt(id_vars=cols_to_keep, value_name='VALUE',var_name='METRIC')
    domain_val['ENTITY_GROUPS'] = domain_val['ENTITY']
    domain_val.loc[domain_val['METRIC'] == 'VALUE', 'LABEL_ORDER'] = 99999999
    domain_val.loc[domain_val['METRIC'] == 'VALUE_D-1', 'LABEL_ORDER'] = 0
    domain_val.loc[domain_val['METRIC'] == 'VALUE', 'METRIC'] = domain_val['SCENARIO']
    domain_val.loc[domain_val['METRIC'] == 'VALUE_D-1', 'METRIC'] = (pd.to_datetime(domain_val['SCENARIO'], format='%d/%m/%Y') + timedelta(days=-1)).dt.strftime('%d/%m/%Y')
    
    cols_to_rename = {'METRIC':'LABEL', 'ENTITY_GROUPS': 'ENTITY','KPI':'UPPER_FILTER_R'}
    domain_val = domain_val.drop('ENTITY',axis=1).rename(index=str, columns=cols_to_rename)
    domain_val['PRECISION'] = ',.0f'

    #--Variation
    #filters = ['Europe','Asia','Africa','North America','South America','Oceania','Other','To be affected']
    domain_var = df.copy()[['ENTITY_GROUPS','ENTITY','SCENARIO','KPI','VARV','VARP']][df['ENTITY_GROUPS']!='WORLDWIDE']#[df['ENTITY_GROUPS'].isin(filters)]
    cols_to_rename = {'VARV':'VALUE', 'VARP': 'VAR'}
    domain_var = domain_var.rename(index=str, columns=cols_to_rename).reset_index(drop=True)
    domain_var = domain_var[domain_var['VALUE'] !=0 ]
    
    #-- Get the first 10
    filters = domain_var[['ENTITY_GROUPS','SCENARIO','KPI']].drop_duplicates().values.tolist()
    domain_w = pd.DataFrame()
    for f in filters:
        tmp_df = domain_var[(domain_var['ENTITY_GROUPS'] == f[0]) & (domain_var['SCENARIO'] == f[1]) & (domain_var['KPI'] == f[2])].sort_values(by=["VALUE","ENTITY"],ascending=False).reset_index(drop=True)
        tmp_df['LABEL_ORDER'] = tmp_df.index.astype(int) + 1
        if len(tmp_df) > 9:
            tmp_df.loc[tmp_df.index > 9, 'ENTITY'] = 'Others'
            tmp_df.loc[tmp_df.index > 9, 'LABEL_ORDER'] = 11
            cols_to_group = ['ENTITY_GROUPS','ENTITY','SCENARIO','KPI','LABEL_ORDER']
            tmp_df = tmp_df.groupby(cols_to_group, as_index=False).agg({'VALUE':'sum'}).reset_index(drop=True)
        tmp_df['PRECISION'] = '+,.0f'
        domain_w = domain_w.append(tmp_df)
        
    cols_to_rename = {'ENTITY':'LABEL', 'ENTITY_GROUPS': 'ENTITY','KPI':'UPPER_FILTER_R'}
    domain_w = domain_w.rename(index=str, columns=cols_to_rename)
    domain_w['GROUPS'] = domain_w['LABEL']
    
    filters = ['Europe','Asia','Africa','North America','South America','Oceania','Other','To be affected']
    cols_to_rename = {'LABEL':'GROUPS', 'ENTITY':'LABEL'}
    domain_w2 = domain_w.copy()[domain_w['ENTITY'].isin(filters)].drop('GROUPS',axis=1).rename(index=str, columns=cols_to_rename)
    domain_w2['ENTITY'] = 'WORLDWIDE'
    
    domain = pd.concat([domain_val,domain_w,domain_w2],axis=0)

    #-- Save df
    df_to_csv_mongo(domain,domain_num)
    df_collection = MONGO_DB[DB_APP][domain_num]
    df_collection.create_index([("ENTITY", DESCENDING), ("SCENARIO", DESCENDING), ("UPPER_FILTER_R", ASCENDING)])
    domain = optimize(domain, ['LAST_UPDATE'])
    return domain

domain_106(db_all, '106')
# domain106 = domain_106(db_all, '106')
# domain106

<timed exec>:12: FutureWarning: This dataframe has a column name that matches the 'value_name' column name of the resultiing Dataframe. In the future this will raise an error, please set the 'value_name' parameter of DataFrame.melt to a unique name.


106 successfully saved in 106
Dataframe 106 successfully save in database app-wsr in MongoDB. Time: --- 68.94032001495361 secnds ---
CPU times: user 1min 23s, sys: 1.29 s, total: 1min 24s
Wall time: 2min 29s


,ENTITY,SCENARIO,UPPER_FILTER_R,LABEL,VALUE,LABEL_ORDER,PRECISION,VAR,GROUPS
0,Algeria,25/10/2020,Confirmed,25/10/2020,56143.0,100000000.0,",.0f",NaN,NaN
1,Algeria,26/10/2020,Confirmed,26/10/2020,56419.0,100000000.0,",.0f",NaN,NaN
2,Algeria,27/10/2020,Confirmed,27/10/2020,56706.0,100000000.0,",.0f",NaN,NaN
3,Algeria,28/10/2020,Confirmed,28/10/2020,57026.0,100000000.0,",.0f",NaN,NaN
4,Algeria,29/10/2020,Confirmed,29/10/2020,57332.0,100000000.0,",.0f",NaN,NaN
...,...,...,...,...,...,...,...,...,...
1,WORLDWIDE,27/10/2020,Active cases,To be affected,4.0,2.0,"+,.0f",0.432432,Lesotho
0,WORLDWIDE,02/11/2020,Active cases,To be affected,5.0,1.0,"+,.0f",125.000000,Solomon Islands
1,WORLDWIDE,02/11/2020,Active cases,To be affected,-5.0,2.0,"+,.0f",-0.535332,Lesotho
0,WORLDWIDE,06/11/2020,Active cases,To be affected,-32.0,1.0,"+,.0f",-3.437164,Lesotho


In [18]:
%run "map-chart_us.ipynb"

Successfully connected to MongoDB
Data not updated !


UsageError: Line magic function `%stop` not found.


NameError: name 'df_us' is not defined

NameError: name 'df_us' is not defined

In [19]:
%run "map-chart_world.ipynb"

Successfully connected to MongoDB
2020-11-09
              ENTITY            LABEL            KPI     VALUE  VALUE_D-1  \
0        Afghanistan      Afghanistan      Confirmed   0.00000    0.00000   
1        Afghanistan      Afghanistan      Confirmed   0.00000    0.00000   
2        Afghanistan      Afghanistan      Confirmed   0.00000    0.00000   
3        Afghanistan      Afghanistan      Confirmed   0.00000    0.00000   
4        Afghanistan      Afghanistan      Confirmed   0.00000    0.00000   
...              ...              ...            ...       ...        ...   
1254427    WORLDWIDE  Solomon Islands  Recovery Rate  30.76923   30.76923   
1254428    WORLDWIDE  Solomon Islands  Recovery Rate  30.76923   30.76923   
1254429    WORLDWIDE  Solomon Islands  Recovery Rate  30.76923   30.76923   
1254430    WORLDWIDE  Solomon Islands  Recovery Rate  30.76923   30.76923   
1254431    WORLDWIDE  Solomon Islands  Recovery Rate  30.76923   30.76923   

         VAR  VARP    SCENARIO

UsageError: Line magic function `%stop` not found.
UsageError: Line magic function `%stop` not found.
